# neural network

### Import libraries

In [1]:
import sys, os

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import time
import copy

In [3]:
from src.neural_network import utils
from torchvision import models

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, output_size)
        )
        
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

### Configuration

In [5]:
#
# cpu or gpu
#
#device= torch.device("cuda")
device= torch.device("cpu")
print(device)

#
# dataset
#
train_dataset = "./imdb_labeled"
test_dataset = "./fgnet-labeled"

#
# net
#
# input of networks
channels = 3
img_pixels = (128,128)

# output of networks
classes = 100

# the test is much faster than train, but not negligable (say <15% of the whole time)
# 74 training imgs, 10 test imgs

# 0, 1 class has 100%, very fast
#net = MLP(channels * img_pixels[0] * img_pixels[1], 512, 512, classes)

# 5, from 0.005 to 0.02, 12s per epoch
net = models.resnet18(num_classes=classes)

# 0, 1 class has 100%, 4s per epoch
#net = models.alexnet(num_classes=classes)

# 10, all near 0.01, 2min+ per epoch
#net = models.vgg19(num_classes=classes)

# 10, all near 0.01, 2min+ per epoch
#net = models.vgg19_bn(num_classes=classes)

# 7, from 0.0096 to 0.012, 6s per epoch
#net = models.squeezenet1_0(num_classes=classes)

# 6, from 0.005 to 0.015, 100s per epoch
#net = models.densenet161(num_classes=classes)

# tricky to run, maybe later (maybe never)
# depends on scipy
# can not run on (256, 256) pixel, but work at (512, 512)
# 'InceptionOutputs' object has no attribute 'log_softmax'
#net = models.inception_v3(num_classes=classes)

# tricky to run, maybe later (maybe never)
# can not run on (256, 256) pixel, but work at (512, 512)
#net = models.googlenet(num_classes=classes)

# 10, all near 0.01, 3s per epoch
#net = models.shufflenet_v2_x1_0(num_classes=classes)

# 10, all exactly 0.01, 8s per epoch
#net = models.mobilenet_v2(num_classes=classes)

# 10, all near 0.01, 30s per epoch
#net = models.resnext50_32x4d(num_classes=classes)

# 0, 1 class takes 0.97, 1min per epoch
#net = models.wide_resnet50_2(num_classes=classes)

# 10, all exactly 0.01, 8s per epoch
#net = models.mnasnet1_0(num_classes=classes)

#
# training detail
#
# CONFIRM: whether lr is dynamically determined by pytorch
lr = 0.01
num_epochs = 15
batch_size = 256

#
# misc
#
#print(net)
utils.display_num_param(net)

cpu
There are 11227812 (11.23 million) parameters in this neural network


### Setting train & eval & test dataset

In [6]:
import torchvision
from torchvision import transforms

transform = transforms.Compose([
    # TODO: check this more later
    #transforms.Grayscale(num_output_channels=3),
    transforms.Resize(img_pixels),
    transforms.ToTensor()])

img_data_train = torchvision.datasets.ImageFolder(root=train_dataset, transform=transform)
print('train data len:', len(img_data_train))
data_loader_train = torch.utils.data.DataLoader(img_data_train, batch_size=batch_size,shuffle=True)
print(len(data_loader_train))

img_data_val = torchvision.datasets.ImageFolder(root=test_dataset, transform=transform)
print(len(img_data_val))
data_loader_val = torch.utils.data.DataLoader(img_data_val, batch_size=batch_size,shuffle=True)
print(len(data_loader_val))

img_data_test = torchvision.datasets.ImageFolder(root=test_dataset, transform=transform)
data_loader_test = torch.utils.data.DataLoader(img_data_test, batch_size=1,shuffle=True)

dataloaders = {}
dataloaders['train'] = data_loader_train
dataloaders['val'] = data_loader_val

train data len: 60590
237
1002
4


### Helper function to training

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    last = since
    time_elapsed = since

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            time_elapsed = time.time() - last
            last = time.time()
            
            print('{} Loss: {:.4f} Acc: {:.4f} Time: {:.0f}m {:.0f}s'.format(phase, epoch_loss, epoch_acc, time_elapsed // 60, time_elapsed % 60))

            # deep copy the modeltopk
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

### Training

In [ ]:
net = net.to(device)
criterion = nn.CrossEntropyLoss()
# use ADAM, better than SGD
optimizer=torch.optim.Adam(net.parameters(), lr)
net, val_acc_history = train_model(net, dataloaders, criterion, optimizer, num_epochs)

Epoch 0/14
----------
train Loss: 4.0322 Acc: 0.0464 Time: 46m 57s
val Loss: 6.6652 Acc: 0.0110 Time: 0m 12s

Epoch 1/14
----------
train Loss: 3.8682 Acc: 0.0483 Time: 40m 37s
val Loss: 6.0374 Acc: 0.0110 Time: 0m 12s

Epoch 2/14
----------
train Loss: 3.8026 Acc: 0.0504 Time: 40m 40s
val Loss: 6.3261 Acc: 0.0120 Time: 0m 12s

Epoch 3/14
----------
train Loss: 3.7610 Acc: 0.0542 Time: 40m 44s
val Loss: 5.9585 Acc: 0.0100 Time: 0m 12s

Epoch 4/14
----------
train Loss: 3.7226 Acc: 0.0552 Time: 40m 37s
val Loss: 6.1886 Acc: 0.0120 Time: 0m 12s

Epoch 5/14
----------
train Loss: 3.6841 Acc: 0.0576 Time: 40m 43s
val Loss: 5.9110 Acc: 0.0190 Time: 0m 12s

Epoch 6/14
----------
train Loss: 3.6441 Acc: 0.0579 Time: 40m 39s
val Loss: 5.9983 Acc: 0.0180 Time: 0m 12s

Epoch 7/14
----------
train Loss: 3.6022 Acc: 0.0600 Time: 40m 49s
val Loss: 5.9817 Acc: 0.0120 Time: 0m 12s

Epoch 8/14
----------
train Loss: 3.5596 Acc: 0.0655 Time: 40m 43s
val Loss: 5.8782 Acc: 0.0110 Time: 0m 13s

Epoch 9/14

### Evaluation on test data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def show_prob_age(label, p):

    p=p.data.squeeze().numpy()

    ft=15
    #p=p.data.squeeze().numpy()
    y_pos = np.arange(len(p))*1.2
    target=2
    width=0.9
    col= 'blue'
    #col='darkgreen'

    plt.rcdefaults()
    fig, ax = plt.subplots()

    # the plot
    ax.barh(y_pos, p, width , align='center', color=col)

    ax.set_xlim([0, 1.3])
    #ax.set_ylim([-0.8, len(p)*1.2-1+0.8])

    # y label
    ax.set_yticks(y_pos)
    ax.set_yticklabels(label, fontsize=ft)
    ax.invert_yaxis()
    #ax.set_xlabel('Performance')
    #ax.set_title('How fast do you want to go today?')

    # x label
    ax.set_xticklabels([])
    ax.set_xticks([])
    #x_pos=np.array([0, 0.25 , 0.5 , 0.75 , 1])
    #ax.set_xticks(x_pos)
    #ax.set_xticklabels( [0, 0.25 , 0.5 , 0.75 , 1] , fontsize=15)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_linewidth(4)

    for i in range(len(p)):
        str_nb="{0:.2f}".format(p[i])
        ax.text( p[i] + 0.05 , y_pos[i] ,str_nb ,
                 horizontalalignment='left', #verticalalitopkgnment='center',
                 transform=ax.transData, color= col,fontsize=ft)

    plt.show()


In [ ]:
# switch to eval mode
net.eval()

label_dict = dict(map(reversed, img_data_test.class_to_idx.items()))

# display tensor
for input, label in data_loader_test:
    #print(input.size())
    image = torch.tensor(input.tolist()[0])
    trans = transforms.ToPILImage()
    #print(image.size())
    plt.imshow(trans(image))
    output = net(input)
    probs= F.softmax(output, dim=1)
    top10 = torch.topk(probs[0], 10)
    top10value = top10.values
    print(top10value)
    top10ages = [ label_dict[a] for a in top10.indices.tolist() ]
    print(top10value, top10ages)
    show_prob_age(top10ages, top10value.cpu())
    break